In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from time import sleep
import os
import random
from sklearn.metrics import confusion_matrix, classification_report
import cv2

In [2]:
DATADIR = r".\DL_TRAIN_DATA\Images"
CATEGORIES = ["Books", "Boxes", "Cups"]
IMG_SIZE = 100

In [3]:
def make_train_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                train_data.append([resized_img_array, class_num])
            except Exception as e: 
                pass

In [4]:
train_data = []
make_train_data()
print(len(train_data))

24


In [5]:
random.shuffle(train_data)

In [6]:
X = []
y = []

for features, label in train_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X = X/255.0
y = np.array(y)

In [7]:
class tooGood(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get("accuracy") > 0.9):
            print("\n Desired accuracy reached; training ended.")
            self.model.stop_training = True

In [8]:
callbacks = tooGood()

In [9]:
cnn_model = models.Sequential([
    
    layers.Conv2D(filters = 64, kernel_size = (3,3), activation = "relu", input_shape = X.shape[1:]),
    layers.MaxPooling2D((2,2)),
    
    #layers.Conv2D(filters = 64, kernel_size = (3,3), activation = "relu"),
    #layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(64, activation = "relu"),
    layers.Dense(3, activation = "softmax")
    
    #Maybe add another Dense 1 and activation? Maybe activation sigmoid? 
    
])

In [10]:
cnn_model.compile(optimizer = "adam", 
             loss = "sparse_categorical_crossentropy", 
            #Maybe do binary_crossentropy? if it's greyscale... 
             metrics = ["accuracy"])

In [34]:
cnn_model.fit(X, y, batch_size = 1, epochs = 10) #, callbacks = [callbacks])

Epoch 1/10
24/24 [==============================] - 2s 64ms/step - loss: 8.2709e-05 - accuracy: 1.0000
Epoch 2/10
24/24 [==============================] - 1s 54ms/step - loss: 8.0637e-05 - accuracy: 1.0000
Epoch 3/10
24/24 [==============================] - 1s 53ms/step - loss: 7.6517e-05 - accuracy: 1.0000
Epoch 4/10
24/24 [==============================] - 1s 54ms/step - loss: 7.5316e-05 - accuracy: 1.0000
Epoch 5/10
24/24 [==============================] - 1s 53ms/step - loss: 7.3127e-05 - accuracy: 1.0000
Epoch 6/10
24/24 [==============================] - 1s 53ms/step - loss: 7.3528e-05 - accuracy: 1.0000
Epoch 7/10
24/24 [==============================] - 1s 53ms/step - loss: 7.4566e-05 - accuracy: 1.0000
Epoch 8/10
24/24 [==============================] - 1s 53ms/step - loss: 6.8762e-05 - accuracy: 1.0000
Epoch 9/10
24/24 [==============================] - 1s 54ms/step - loss: 6.5391e-05 - accuracy: 1.0000
Epoch 10/10
24/24 [==============================] - 1s 54ms/step - loss:

In [12]:
DATADIR_TEST = r".\DL_DATA"
CATEGORIES_TEST = ["Books", "Boxes", "Cups"]

In [13]:
def make_test_data():
    for category_test in CATEGORIES_TEST:
        path_test = os.path.join(DATADIR_TEST, category_test)
        class_num_test = CATEGORIES_TEST.index(category_test)
        for img_test in os.listdir(path_test):
            try:
                img_array_test = cv2.imread(os.path.join(path_test, img_test), cv2.IMREAD_GRAYSCALE)
                resized_img_array_test = cv2.resize(img_array_test, (IMG_SIZE, IMG_SIZE))
                test_data.append([resized_img_array_test, class_num_test])
            except Exception as e: 
                pass

In [14]:
test_data = []
make_test_data()
print(len(test_data))

1012


In [15]:
random.shuffle(test_data)

In [26]:
X_test = []
y_test = []

for features, label in test_data:
    X_test.append(features)
    y_test.append(label)

X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_test = X_test/255.0
y_test = np.array(y_test)

print(len(X_test))
print(len(y_test))

1012
1012


In [35]:
cnn_model.evaluate(X_test,y_test)

32/32 [==============================] - 1s 45ms/step - loss: 1.8540 - accuracy: 0.4654


[1.854002833366394, 0.46541503071784973]

In [36]:
y_pred = cnn_model.predict(X_test)
y_pred_labels = [np.argmax(elem) for elem in y_pred]

print("Classification Report: \n", classification_report(y_test, y_pred_labels))

Classification Report: 
               precision    recall  f1-score   support

           0       0.51      0.89      0.65       438
           1       0.00      0.00      0.00       283
           2       0.62      0.28      0.39       291

    accuracy                           0.47      1012
   macro avg       0.38      0.39      0.35      1012
weighted avg       0.40      0.47      0.39      1012

